In [ ]:
import streamlit as st
import pandas as pd
import pymysql
import altair as alt
import folium
import os
import time
import random

from streamlit_folium import folium_static
from geopy.geocoders import Nominatim
from streamlit_navigation_bar import st_navbar

st.set_page_config(page_title="서울시 주유소 대시보드", layout="wide")

# Create 3 columns: left, center, right
col1, col2, col3, col4, col5, col6, col7, col8, col9 = st.columns([3, 5, 3, 5, 3, 5, 3, 5, 3])  # Adjust ratio if needed

with col6:
    st.page_link("pages/_faq.py", label="FAQ")

with col8: 
    st.page_link("pages/_download.py", label = "Downloads")

with col4:
    st.page_link("pages/_graphs.py", label="Oil Price")

with col2:
    st.page_link("pages/_aa.py", label="Home")

# CSS 스타일 적용
st.markdown("""
    <style>
    /* GS칼텍스 */
    span[aria-label="GS칼텍스, close by backspace"] {
        background-color: #48B445 !important;  /* 초록색 */
        color: white !important;
    }
    
    /* HD현대오일뱅크 */
    span[aria-label="HD현대오일뱅크, close by backspace"] {
        background-color: #79B3EA !important;  /* 파란색 */
        color: white !important;
    }
    
    /* S-OIL */
    span[aria-label="S-OIL, close by backspace"] {
        background-color: #FFE501 !important;  /* 노란색 */
        color: black !important;
    }
    
    /* SK에너지 */
    span[aria-label="SK에너지, close by backspace"] {
        background-color: #FF0000 !important;  /* 빨간색 */
        color: white !important;
    }
    
    /* 알뜰주유소 */
    span[aria-label="알뜰주유소, close by backspace"] {
        background-color: #9F468F !important;  /* 보라색 */
        color: white !important;
    }
    
    /* 자가상표 */
    span[aria-label="자가상표, close by backspace"] {
        background-color: #808080 !important;  /* 회색 */
        color: white !important;
    }
    
    /* 호버 효과 */
    span[aria-label*="close by backspace"]:hover {
        opacity: 0.8 !important;
    }
    </style>
""", unsafe_allow_html=True)

# CSV 파일 경로 설정
csv_path = "../crawling/2025-04-10.csv"  # 경로는 상황에 맞게 조정하세요

# # 데이터 불러오기
df = pd.read_csv(csv_path)

conn = pymysql.connect(
        host='192.168.0.45', # DB 주소 (예: '127.0.0.1' 또는 AWS RDS 주소)
        port = 3306,
        user='3team',    # MySQL 사용자
        password='1111',
        db='gas_station',
        charset='utf8'  # 저장할 데이터베이스명
    )

# SQL 쿼리 실행하여 데이터 가져오기
query = """
    SELECT gs.*, b.*
    FROM gas_station gs
    JOIN brand b ON gs.brand_id = b.brand_id
"""
df = pd.read_sql(query, conn)

# 연결 종료
conn.close()

print(df.head())

# 휘발유, 경유 → 숫자형으로 변환 (쉼표 제거 및 에러 무시)
df["gasoline_price"] = pd.to_numeric(df["gasoline_price"].astype(str).str.replace(",", "").str.strip(), errors="coerce")
df["diesel_price"] = pd.to_numeric(df["diesel_price"].astype(str).str.replace(",", "").str.strip(), errors="coerce")

# 필터 옵션
gu_options = ["전체"] + sorted(df["region"].dropna().unique().tolist())
brand_options = sorted(df["brand_name"].dropna().unique())

st.sidebar.image("openoil.png", width=800)
# st.sidebar.header("🔍 필터 옵션")
selected_gu = st.sidebar.selectbox("지역 선택", gu_options)
selected_brand = st.sidebar.multiselect("브랜드 필터", brand_options, default=brand_options)

#체크박스 필터링 된 데이터 출력
# 사이드바 체크박스
st.sidebar.write("부가 옵션")
self_service = st.sidebar.checkbox("셀프 주유소", key="self_checkbox")
car_wash = st.sidebar.checkbox("세차장", key="wash_checkbox")
convenience_store = st.sidebar.checkbox("편의점", key="store_checkbox")
open_24h = st.sidebar.checkbox("24시 운영", key="24h_checkbox")
 
# 필터링 준비

filtered = df.copy()

# NaN 제거 (없으면 필터링 실패함)
filtered = filtered.fillna({
    "self_service": "N",
    "car_wash": "N",
    "convenience_store": "N",
    "hours_24": "N"
})

# 각 체크박스의 상태에 따라 필터링
if self_service:
     filtered = filtered[filtered["self_service"]=="Y"]
if car_wash:
     filtered = filtered[filtered["car_wash"]=="Y"]
if convenience_store:
     filtered = filtered[filtered["convenience_store"]=="Y"]
if open_24h:
     filtered = filtered[filtered["hours_24"]=="Y"]
    
# 사이드바 가격정렬
sort_option = st.sidebar.radio("가격 정렬", ["휘발유 낮은순", "경유 낮은순"])
# 사이드바 가격슬라이더
price_gasoline = st.sidebar.slider ("휘발유 가격" , 0 , 3000, step=10, format="%d원", key="price_gasoline_slider", value = 3000)
price_diesel = st.sidebar.slider ("경유 가격" , 0 , 3000, step=10, format="%d원", key="price_diesel_slider", value = 3000)

  
# 필터 적용

if selected_gu != "전체":
    filtered = filtered[filtered["region"] == selected_gu]

filtered = filtered[filtered["brand_name"].isin(selected_brand)]

if sort_option == "휘발유 낮은순":
    filtered = filtered.sort_values("gasoline_price", ascending=True)
if sort_option == "경유 낮은순":
    filtered = filtered.sort_values("diesel_price", ascending=True)
filtered = filtered[filtered["gasoline_price"] <= price_gasoline]
filtered = filtered[filtered["diesel_price"] <= price_diesel]

# 검색 기능 추가
st.markdown("### 주유소 검색")
search_term = st.text_input("", placeholder="주유소 이름, 주소, 브랜드로 검색", label_visibility="collapsed")

if search_term:
    # 대소문자 구분 없이 검색
    search_filter = (
        filtered["station_name"].str.contains(search_term, case=False, na=False) |
        filtered["address"].str.contains(search_term, case=False, na=False) |
        filtered["brand_name"].str.contains(search_term, case=False, na=False)
    )
    filtered = filtered[search_filter]

# 브랜드별 색상 매핑 함수
def get_brand_color(brand):
    # 브랜드별 고정 색상 매핑
    brand_colors = {
        'GS칼텍스': 'green',
        'S-OIL': 'yellow',
        'SK에너지': 'red',
        'HD현대오일뱅크': 'blue',
        '알뜰주유소': 'orange',
        '코끼리주유소': 'gray',
        '정호주유소': 'gray',
    }
    return brand_colors.get(brand, 'gray')  # 기본값은 회색

# 지도 표시 함수
def show_map(filtered_data):
    # 서울 중심 좌표
    m = folium.Map(
        location=[37.5665, 126.9780], 
        zoom_start=11
    )
    
    # 각 주유소에 대해 마커 추가
    for idx, row in filtered_data.iterrows():
        if pd.notnull(row['latitude']) and pd.notnull(row['longitude']):
            # HTML을 사용하여 팝업 텍스트 포맷팅
            popup_text = f"""
            <div style='font-family: Arial; font-size: 14px;'>
                <b>{row['station_name']}</b><br>
                <span style='color: #666;'>{row['brand_name']}</span><br>
                <span style='color: #666;'>TEL. {row['cst_service']}</span>
                <div style='margin-top: 5px;'>
                    <span style='color: #e74c3c;'>휘발유: {row['gasoline_price']}원</span><br>
                    <span style='color: #3498db;'>경유: {row['diesel_price']}원</span>
                </div>
            </div>
            """
            popup = folium.Popup(popup_text, max_width=300)
            
            folium.Marker(
                [row['latitude'], row['longitude']],
                popup=popup,
                tooltip=row['station_name'],
                icon=folium.Icon(color=get_brand_color(row['brand_name']))
            ).add_to(m)
    
    return m

# 지도 표시
st.write("🗺️ 지도에 표시될 주유소 수:", len(filtered))
map = show_map(filtered)
folium_static(map, width=None, height=600)

# 필터된 데이터 테이블 출력

columns_to_show = ["station_name", "address", "brand_name", "gasoline_price", "diesel_price"]
filtered_display = filtered[columns_to_show]
filtered_display = filtered_display.rename(columns={
    "station_name": "주유소 이름",
    "address": "주소",
    "brand_name": "브랜드",
    "gasoline_price": "휘발유 가격",
    "diesel_price": "경유 가격"
})
# st.subheader(f"📋 {selected_gu}의 주유소 목록")
st.dataframe(filtered_display.reset_index(drop=True), use_container_width=True, hide_index=True)
